In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm

### Sparsity in Matrices

Consider building ratings matrix where we are given with a large set of users who have watched a large set of movies. Naturally, all users watch a handful of movies and all movies are watched by only a handful of users based on their interests. And so, the ratings matrix which comprises of ratings given by a user to a movie, is going to be sparse.

Performing matrix operations on such matrices like inverting the matrix, multiplying two matrices can be computationally costly given their size. However, as these matrices contain information in a small portion of complete data volume (rest are zeros), there are efficient ways of representing/storing this matrix. 

In [2]:
arr = np.array([[1,0,0,0,2], [2,0,4,0,0], [0,0,0,5,0]])
print(arr)

sparsity = np.sum(arr==0) / (arr.shape[0] * arr.shape[1])
print('Sparsity:', sparsity)

[[1 0 0 0 2]
 [2 0 4 0 0]
 [0 0 0 5 0]]
Sparsity: 0.6666666666666666


### Sparse Matrix Representations

Some of the representations are as follows:

1. **Compressed Sparse Row (CSR)** format: 
    1. Processes non-zero elements in data row-wise
    2. Outputs three arrays:
        1. Data: Non-zero values
        2. Indices: Column indices
        3. Indptr: 
            1. Index of starting element of each row
            2. Last element is number of non-zero elements
            3. Length - number of rows + 1
            4. Is increasing in nature
            5. For zero row, index of subsequent non-zero element

2. **Compressed Sparse Column (CSC)** format: 
    1. Processes non-zero elements in data column-wise
    2. Outputs three arrays:
        1. Data: Non-zero values
        2. Indices: Row indices
        3. Indptr: 
            1. Index of starting element of each column
            2. Last element is number of non-zero elements
            3. Length - number of columns + 1
            4. Is increasing in nature
            5. For zero column, index of subsequent non-zero element

3. **COOrdinate format (COO)** format: 
    1. Also called as triplet format
    2. Typically after getting COO format, we convert to CSR/CSC matrix for faster algebraic operations.

4. Another format: Three lists which need not be of same length comprising of the following:
    1. <code>value</code>: Non-zero values
    2. <code>column_idx</code>: Column indices of non-zero values 
    3. <code>row_idx</code>: Row indices from value list for starting element of each row

These representations can be created by storing the data in form of key-value pairs in dictionary or lists of (values, row indices and column indices). However, the non-zero elements can be in any order, need not ordered row-wise or column-wise.

In [3]:
def get_csr_representation(arr):
    n_rows, n_cols = arr.shape
    matrix_dict = {}

    for row in range(n_rows):
        for col in range(n_cols):
            if arr[row][col] != 0:
                matrix_dict[(row, col)] = arr[row][col]
                item = (row, col, arr[row][col])
    return matrix_dict

def get_csc_representation(arr):
    n_rows, n_cols = arr.shape
    matrix_dict = {}
    
    for col in range(n_cols):
        for row in range(n_rows):
            if arr[row][col] != 0:
                matrix_dict[(row, col)] = arr[row][col]
                item = (row, col, arr[row][col])
    return matrix_dict    

In [4]:
csr_format = get_csr_representation(arr)
print(csr_format,'\n')

row_idxs = np.array(list(csr_format.keys()))[:,0]
col_idxs = np.array(list(csr_format.keys()))[:,1]
values = list(csr_format.values())

S = sp.csr_matrix((values, (row_idxs, col_idxs)))
print('Data:', S.data)
print('Column Indices:', S.indices)
print('Row indices:', S.indptr)

{(0, 0): 1, (0, 4): 2, (1, 0): 2, (1, 2): 4, (2, 3): 5} 

Data: [1 2 2 4 5]
Column Indices: [0 4 0 2 3]
Row indices: [0 2 4 5]


In [5]:
csc_format = get_csc_representation(arr)
print(csc_format, '\n')

row_idxs = np.array(list(csc_format.keys()))[:,0]
col_idxs = np.array(list(csc_format.keys()))[:,1]
values = list(csc_format.values())

S = sp.csc_matrix((values, (row_idxs, col_idxs)))
print('Data:', S.data)
print('Column Indices:', S.indices)
print('Row indices:', S.indptr)

{(0, 0): 1, (1, 0): 2, (1, 2): 4, (2, 3): 5, (0, 4): 2} 

Data: [1 2 4 5 2]
Column Indices: [0 1 1 2 0]
Row indices: [0 2 2 3 4 5]


In [6]:
def make_csr_arrays(arr):
    n_rows, n_cols = arr.shape
    vals = []
    col_idxs = []
    row_idxs = []

    val_idx = -1
    for row in range(n_rows):
        start = 1
        start_idx = val_idx + 1
        
        for col in range(n_cols):
            val = arr[row][col]

            if val != 0:
                vals.append(val)
                val_idx += 1
                col_idxs.append(col)

                if start:
                    start_idx = val_idx
                    start = 0 
        row_idxs.append(start_idx)
    row_idxs.append(len(vals))
    return vals, col_idxs, row_idxs

In [7]:
arr = np.array([[0, 0, 0],
                [8, 0, 0],
                [0, 5, 4],
                [0, 0, 0],
                [0, 0, 7]])

vals, col_idxs, row_idxs = make_csr_arrays(arr)
print('Data:', vals)
print('Column Indices:', col_idxs)
print('Row indices:', row_idxs)

Data: [8, 5, 4, 7]
Column Indices: [0, 1, 2, 2]
Row indices: [0, 0, 1, 3, 3, 4]


### ML1M Dataset

Now we will try to build sparse matrices and perform computations in sparse format for RecWalk algorithm. We observe that: 
1. Performing computations in sparse matrix format (CSR/CSC/COO) 
2. **Quantization:** Choosing efficient datatypes (low-precision formats) 

optimizes run-time and reduces memory overhead.

In [8]:
alpha = 0.2
K = 20
eta = 0.8
n_iter = 50

In [9]:
# preprocessing movies data
with open('ml-1m/movies.dat','r', encoding="ISO-8859-1") as f:
    data = f.read().splitlines()

data = list(map(lambda x: x.split('::'), data))
movies_df = pd.DataFrame(data, columns = ['movieID','title','genres'])
movies_df['movieID'] = movies_df['movieID'].astype('int')

# preprocessing ratings data
with open('ml-1m/ratings.dat','r', encoding="ISO-8859-1") as f:
    data = f.read().splitlines()

data = list(map(lambda x: x.split('::'), data))
ratings_df = pd.DataFrame(data, columns = ['userID','movieID','rating','timestamp'])
ratings_df[['userID','movieID','rating']] = ratings_df[['userID','movieID','rating']].astype('int')

In order to keep the matrices precise, we will map userIDs and movieIDs to unique IDs starting from zero. This means if there are 6000 users with maximum userID as 9000, we have 6000 rows and not 9000 rows, one row per user. Likewise for movieIDs. 

In [10]:
users_list = ratings_df['userID'].unique().tolist()
movies_list = ratings_df['movieID'].unique().tolist()

n_users = len(users_list)
n_movies = len(movies_list)

# relabelling userIDs
userID2idx = dict(zip(users_list, range(n_users)))
idx2userID = {v:k for k, v in userID2idx.items()}
ratings_df['userID'].replace(userID2idx, inplace=True)

# relabelling movieIDs
movieID2idx = dict(zip(movies_list, range(n_movies)))
idx2movieID = {v:k for k, v in movieID2idx.items()}
ratings_df['movieID'].replace(movieID2idx, inplace=True)

print('Number of Users:', n_users) 
print('Number of Movies:', n_movies)

Number of Users: 6040
Number of Movies: 3706


In [11]:
def get_sparse_matrices(interaction_dict, ratings_dict):
    values = []
    row_idxs = []
    col_idxs = []

    userIDs = list(interaction_dict.keys())
    for userID in tqdm(userIDs):
        movieIDs = interaction_dict[userID].tolist()
        ratings = ratings_dict[userID].tolist()
        row_idxs += [userID] * len(movieIDs)
        col_idxs += movieIDs
        values += ratings

    assert len(values) == len(row_idxs) == len(col_idxs)   
    ratings_mat = sp.csr_matrix((values, (row_idxs, col_idxs)))
    R = sp.csr_matrix(([1] * len(row_idxs), (row_idxs, col_idxs)))

    # since W is not sparse, keeping it as numpy array
    W = cosine_similarity(ratings_mat.T, ratings_mat.T, dense_output=True)
    return R, W

In [12]:
# mapping users to their rated movies
interaction_dict = dict(ratings_df.groupby('userID')['movieID'].apply(lambda x: np.array(x)))

# mapping users to their ratings
ratings_dict = dict(ratings_df.groupby('userID')['rating'].apply(lambda x: np.array(x)))

R, W = get_sparse_matrices(interaction_dict, ratings_dict)
R

100%|██████████| 6040/6040 [00:00<00:00, 79982.81it/s]


<6040x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [13]:
M1 = sp.csr_matrix(np.zeros((n_users, n_users), dtype='bool'))
M2 = sp.csr_matrix(np.zeros((n_movies, n_movies), dtype='bool'))
AG = sp.vstack((sp.hstack((M1, R)), sp.hstack((R.T, M2))))

# converting into CSR from COO format
AG = sp.csr_matrix(AG)
AG

<9746x9746 sparse matrix of type '<class 'numpy.int64'>'
	with 2000418 stored elements in Compressed Sparse Row format>

In [14]:
# taking transpose of CSR matrix converts it into CSC format
one = sp.csr_matrix(np.ones(n_users + n_movies, dtype='bool')).transpose()

values = (AG @ one).data
row_idxs = list(range(AG.shape[0]))
col_idxs = list(range(AG.shape[0]))

# inverse calculated through LU decomposition
# LU decomposition requires CSC format
diag_sp_matrix = sp.csc_matrix((values, (row_idxs, col_idxs)))

# inverse of diagonal matrix is diagonal matrix
# expected to be sparse
# CSC format
inv_sp_matrix = sp.linalg.inv(diag_sp_matrix)

# multiplication is commutative
# can multiply CSC with CSR or CSR with CSC
H = inv_sp_matrix @ AG
H

<9746x9746 sparse matrix of type '<class 'numpy.float64'>'
	with 2000418 stored elements in Compressed Sparse Column format>

In [15]:
def get_M_mat(W):
    I = sp.csr_matrix(np.identity(n_users, dtype='bool'))
    N1 = sp.csr_matrix(np.zeros((n_users, n_movies), dtype='bool'))
    N2 = sp.csr_matrix(np.zeros((n_movies, n_users), dtype='bool'))

    W_inf = max(np.sum(W, axis=1))
    MI = W / W_inf + np.diag(1 - np.sum(W, axis=1) / W_inf)

    M = sp.vstack((sp.hstack((I, N1)), sp.hstack((N2, MI))))
    return M

M = get_M_mat(W)
M

<9746x9746 sparse matrix of type '<class 'numpy.float64'>'
	with 11319694 stored elements in COOrdinate format>

In [16]:
P = alpha * H + (1 - alpha) * M
P

<9746x9746 sparse matrix of type '<class 'numpy.float64'>'
	with 13320112 stored elements in Compressed Sparse Column format>

In [17]:
def calculate_non_sparsity(arr):
    non_sparsity = len(arr.data) / (arr.shape[0] * arr.shape[1]) * 100
    return non_sparsity

print('Non-Sparsity in following matrices:\n')
print('R:', round(calculate_non_sparsity(R), 2), '%')
print('AG:', round(calculate_non_sparsity(AG), 2), '%')
print('H:', round(calculate_non_sparsity(H), 2), '%')
print('M:', round(calculate_non_sparsity(M), 2), '%')
print('P:', round(calculate_non_sparsity(P), 2), '%')

Non-Sparsity in following matrices:

R: 4.47 %
AG: 2.11 %
H: 2.11 %
M: 11.92 %
P: 14.02 %


In [18]:
def recwalk_Kstep(P, K):
    M1 = np.identity(n_users, dtype='bool')
    M2 = np.zeros((n_users, n_movies), dtype='bool')
    M3 = np.zeros((n_movies, n_users), dtype='bool')
    M4 = np.zeros((n_movies, n_movies), dtype='bool')
    teleport_dist = np.vstack((np.hstack((M1, M2)), np.hstack((M3, M4))))
    
    T = P.transpose().toarray()
    state = teleport_dist.copy()
    for _ in tqdm(range(K)):
        state = T @ state
    
    recommendations = state[:n_users, n_users:]
    return recommendations

In [19]:
def random_walk(P_user, initial_state, n_iter):
    new_initial_state = initial_state
    NORM = []
    for i in tqdm(range(n_iter)):
        final_state = np.dot(np.transpose(P_user), new_initial_state)
        prev_initial_state = new_initial_state
        new_initial_state = final_state
        L1 = np.linalg.norm(new_initial_state-prev_initial_state, ord=1)
        NORM.append(L1)

        if i!=0 and (i+1)%5==0:
            print(f'L1 Norm at {i+1} iteration..', L1)        
        if np.allclose(new_initial_state, prev_initial_state):
            print(f'Converged at {i+1} iterations..')
            break
    return final_state
    
def recwalk_PR(P, eta, n_iter):
    M1 = np.identity(n_users, dtype='bool')
    M2 = np.zeros((n_users, n_movies), dtype='bool')
    M3 = np.zeros((n_movies, n_users), dtype='bool')
    M4 = np.zeros((n_movies, n_movies), dtype='bool')
    teleport_dist = np.vstack((np.hstack((M1, M2)), np.hstack((M3, M4))))

    random_surf = np.ones((n_users + n_movies, n_users + n_movies), dtype='bool') / (n_users + n_movies)
    
    print('Running random walk..')
    P_user = P.toarray() * eta + (1 - eta) * random_surf
    limiting_state = random_walk(P_user, teleport_dist, n_iter)
    
    recommendations = limiting_state[:n_users, n_users:]
    return recommendations

In [20]:
recommendations = recwalk_Kstep(P, K)

100%|██████████| 20/20 [02:20<00:00,  7.02s/it]


In [21]:
recommendations = recwalk_PR(P, eta, n_iter)

Running random walk..


  8%|▊         | 4/50 [00:39<07:25,  9.68s/it]

L1 Norm at 5 iteration.. 0.1779545912263687


 18%|█▊        | 9/50 [01:28<06:45,  9.90s/it]

L1 Norm at 10 iteration.. 0.02816183383724494


...

L1 Norm at 40 iteration.. 3.788464243166721e-07


 88%|████████▊ | 44/50 [07:35<01:04, 10.78s/it]

L1 Norm at 45 iteration.. 5.697842367180197e-08


 88%|████████▊ | 44/50 [07:46<01:03, 10.60s/it]

Converged at 45 iterations..
